In [ ]:
from facebook_scraper import get_posts
import json
import pandas as pd

In [ ]:
def scrape_comments(fb_group_list, pages, cookies_file, options, words_list):
    """
    Made this dataframe based on: https://github.com/kevinzg/facebook-scraper
    
    Args:
        fb_group_list(str, list): a list of Facebook pages to scrape, must be public
        pages(int): number of pages to collect
        cookies_file(str): file location of cookies data
        options(dict): dictionary of options for collection, dictionary options can be found in the link above
        words_list(str, list): list of words to filter posts by
        
    Returns:
        DataFrame: a DataFrame containing comments and associated information from posts scraped. Posts scraped are gathered from 'fb_group_list' posts that 
                   that contain words inputted in 'words_list'
    """
    # If content in the 'comment_text' column is the commenters name it is most likely because they commented with just a picture and no text
    post_list = []
    df_comments = pd.DataFrame()
    for fb_group in fb_group_list:
        for post in get_posts(fb_group, pages = pages, cookies = cookies_file, options = options):
            post['text'] = post['text'].split('\n',1)[0]
            words = words_list
            comments_list = []
            replies_list = []
            
            if any(word in post['text'] for word in words) == True:
                print(post['text']+'\n')
                post_list.append(post)
                x = 0
                #comments_list = []
                
                while x < len(post_list):
                    #print(post['text'])
                    comments_list = post_list[x]['comments_full']
                    x+=1
                    
                    if comments_list != []:
                        i = 0
                        
                        while i < len(comments_list):
                            comments_list[i]['comment_text'] = comments_list[i]['comment_text'].split('\n',1)[0]
                            replies_list = comments_list[i]['replies']
                            i+=1
                            
                            if replies_list != []:
                                z = 0
                                
                                while z < len(replies_list):
                                    replies_list[z]['comment_text'] = replies_list[z]['comment_text'].split('\n',1)[0]
                                    z+=1
                            
            df_comments = df_comments.append(comments_list)
    return df_comments

In [ ]:
# Initialize function parameters 
fb_group_list = ['']
cookies = ('')
words_list = [']

In [ ]:
scrape_comments(fb_group_list, 50, cookies, {"comments": True, "allow_extra_requests": True, 'posts_per_page': 1}, words_list)

In [ ]:
df_comments = scrape_comments(fb_group_list, 20, cookies, {"comments": True, "allow_extra_requests": True, 'posts_per_page': 1}, words_list)

In [ ]:
df_comments

In [ ]:
# Scraper sometimes returns duplicate posts. This removes instances of the same column that appear more than once
df_comments.drop_duplicates(subset = 'comment_id',
                     keep = 'first', inplace = True)

In [ ]:
df_comments.to_csv('df_comments.csv')